<a href="https://colab.research.google.com/github/agmeadows/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-10 03:16:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-10 03:16:31 (6.44 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://msu-dataviz.s3.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [26]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     176073|             1|
|   20992492|             1|
|    1397686|             1|
|   50371142|             1|
|     235096|             1|
|   12320806|             1|
|   19707203|             1|
|   15207172|             1|
|     538703|             1|
|   11223135|             1|
|   31447151|             1|
|   35961247|             1|
|   17835519|             1|
|   15404885|             1|
|   10182610|             1|
|   52399092|             1|
|   17133428|             1|
|   27870423|             1|
|   11574168|             1|
|   30685607|             1|
+-----------+--------------+
only showing top 20 rows



In [27]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01075GRAO|ANYSECU QYT KT890...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B004ECO5VA|Holux Mini Smart ...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00M1Y87I0|      sentey ls 4063|
|B002ABX29I|Micca Slim Portab...|
|B00XXA2T2G|Centurion GT1 Car...|
|B00Q5A5PD6|HYUNDAI HCR-580 V...|
|B00VFEK8IY|Z ZTDM Mini Stere...|
|B00L85M8VU|Settonbrothers Ul...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00KJ7MVYW|Fincibo (TM) Appl...|
|B00A6YQQS0|Minisuit Aero Fol...|
|B009S99IYI|PIO TSG1644R 6.5 ...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00OXD61OE|SHE'O® 2x Cool No...|
|B000IBMI1K|RCA 200-Watt Mono...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B00V684NCQ|LSD 8GB Flash Dri...|
|B00A609WOO|eForCity Hybrid S...|
+----------+--------------------+
only showing top 20 rows



In [29]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8MEA6IGAHO0B|   20422322|B00MC4CED8|     217304173| 2015-08-31|
|R31LOQ8JGLPRLK|   40835037|B00OQMFG1Q|     137313254| 2015-08-31|
|R2Y0MM9YE6OP3P|   51469641|B00QERR5CY|      82850235| 2015-08-31|
| RRB9C05HDOD4O|    4332923|B00QUFTPV4|     221169481| 2015-08-31|
|R26I2RI1GFV8QG|   44855305|B0067XVNTG|     563475445| 2015-08-31|
| RY8DDL22YG4R5|    7846966|B00KA6CCVY|     713636156| 2015-08-31|
|R2AT2426ZHFUHH|   21299354|B00MJCDPM2|     754171872| 2015-08-31|
|R3RRXU2R23NMQ9|   28902968|B00ET5AWBY|     508373185| 2015-08-31|
|R250PR8VJUZ62F|    2074438|B00Z9K379C|     759387370| 2015-08-31|
| RBEMQ29WJBHYG|   48701722|B00YO3UYXW|     913911909| 2015-08-31|
|R19VVIUT4BZCMT|    2990247|B011W26BU2|     938399424| 2015-08-31|
|R1DT8JJUQHUKUL|   52946117|B00H8PI78C|     661515294| 2015-08

In [32]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [36]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://msu-dataviz.cwuiowjs38ni.us-east-1.rds.amazonaws.com:5432/reviews_dataviz"
config = {"user":"dataviz", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [37]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [38]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [39]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [40]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [41]:
# Filter vine dataframe where total vote count is > 20
new_df = vine_df.filter("total_votes>20")
new_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

In [43]:
# Refine dataframe where helpful votes divided by total votes is > = 50%
refined_df = new_df.filter("helpful_votes/total_votes>.5")
refined_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

In [49]:
# Create dataframe where customer is part of vine program
vine_prog_df = refined_df.filter("vine=='Y'")
vine_prog_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BOQTLUU3Y4L8|          4|           42|         55|   Y|                N|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|
| REPU28WG1VZUE|          5|          242|        281|   Y|                N|
|R1D6REC9HPJVQY|          4|           31|         41|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [50]:
# Create dataframe where customer is not part of vine program
vine_noprog_df = refined_df.filter("vine=='N'")
vine_noprog_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

In [78]:
# Calculate total reviews for customers in the vine program
total_vine_votes = vine_prog_df.groupby().agg({"total_votes" : "sum"}).withColumnRenamed("sum(total_votes)", "all_votes").collect()[0]["all_votes"]
total_vine_votes

822

In [79]:
# Calculate total 5-star rating for customer in the vine program
five_star_prog_votes = vine_prog_df.groupby("star_rating").agg({"star_rating" : "count"}).withColumnRenamed("count(star_rating)", "5-star reviews").filter("star_rating=='5'").collect()[0]["5-star reviews"]
five_star_prog_votes

1

In [97]:
# Percentage of 5-star votes for vine program
vine_five_prcnt = (five_star_prog_votes / total_vine_votes) * 100
print("Percentage of 5-star reviews for vine votes =", vine_five_prcnt)

Percentage of 5-star reviews for vine votes = 0.12165450121654502


In [92]:
# Calculate total reviews for customers not in the vine program
total_novine_votes = vine_noprog_df.groupby().agg({"total_votes" : "sum"}).withColumnRenamed("sum(total_votes)", "all_votes").collect()[0]["all_votes"]
total_novine_votes

49155

In [93]:
# Calculate total 5-star rating for customer not in the vine program
five_star_noprog_votes = vine_noprog_df.groupby("star_rating").agg({"star_rating" : "count"}).withColumnRenamed("count(star_rating)", "5-star reviews").filter("star_rating=='5'").collect()[0]["5-star reviews"]
five_star_noprog_votes

495

In [96]:
# Percentage of 5-star votes for non vine program
novine_five_prcnt = (five_star_noprog_votes / total_novine_votes) * 100
print("Percentage of 5-star reviews for none vine votes =", novine_five_prcnt)

Percentage of 5-star reviews for none vine votes = 1.007018614586512
